# 02. Conference Top Authors By Static Rank    
1. Aggregate static rank and publication for each author  
1. Create two ranks based on different methods to display on the x and y axis for comparison  

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as sf


rootpath = 'wasbs://mag-2018-09-27@magtrainingsource.blob.core.windows.net/mag/'
outputDir = '~/output/pyspark/'
targetConf = 'WWW'
n_top = 20

In [ ]:
# start Spark context
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


# First find the conference series id by matching conference short name
ConferenceSeries = spark.read.load(rootpath + "ConferenceSeries.txt", format="csv", sep="\t") \
.toDF("ConferenceSeriesId", "Rank", "NormalizedName", "DisplayName", "PaperCount", 
      "CitationCount", "CreatedDate") \
.where(sf.col("NormalizedName").isin(targetConf)) \
.select("ConferenceSeriesId")


# Get all conference papers by conference series Id
Papers = spark.read.load(rootpath + "Papers.txt", format="csv", sep="\t") \
.toDF("PaperId", "Rank", "Doi", "DocType", "PaperTitle", "OriginalTitle", "BookTitle", 
      "Year", "Date", "Publisher", "JournalId", "ConferenceSeriesId", "ConferenceInstanceId", 
      "Volume", "Issue", "FirstPage", "LastPage", "ReferenceCount", "CitationCount", 
      "EstimatedCitationCount", "CreatedDate") \
.join(ConferenceSeries, "ConferenceSeriesId", 'inner') \
.selectExpr("PaperId as PaperId", "Rank * -0.001 as LogProbRank")


# Get all [conference paper] -> [conference author] relationships
Conf_Authors = spark.read.load(rootpath + "PaperAuthorAffiliations.txt", format="csv", sep="\t") \
.toDF("PaperId", "AuthorId", "AffiliationId", "AuthorSequenceNumber", "OriginalAffiliation") \
.join(Papers, "PaperId", 'inner') \
.select("AuthorId", "LogProbRank")


# Aggregate static rank and publication for each author
Authors = spark.read.load(rootpath + "Authors.txt", format="csv", sep="\t") \
.toDF("AuthorId", "Rank", "NormalizedName", "DisplayName", "LastKnownAffiliationId", 
      "PaperCount", "CitationCount", "CreatedDate") \
.join(Conf_Authors, "AuthorId", 'inner') \
.groupby("DisplayName") \
.agg(sf.sum("LogProbRank").alias("LogProbRank")) \
.select("DisplayName", "LogProbRank")

PublicationCounts = Authors.groupby("DisplayName") \
.count() \
.selectExpr("DisplayName as DisplayName", "count as PublicationCount")

Authors = Authors.join(PublicationCounts, "DisplayName", 'inner')


# Create two ranks based on different methods to display on the x and y axis for comparison
Conf_TopAuthors = Authors


# Stop Spark context
sc.stop()